# Анализ поведения пользователей мобильного приложения

**Задача:**
Подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего места.

**Ход исследования:**
- Изучение и предобработка данных.
- Изучение воронки продаж.
- Исследование результатов A/A/B-эксперимента.
- Подготовка презентации.


**Данные:** 
- Файл **/datasets/logs_exp.csv**.


**Структура /datasets/logs_exp.csv**:
- **EventName** — название события;
- **DeviceIDHash** — уникальный идентификатор пользователя;
- **EventTimestamp** — время события;
- **ExpId** — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.


# Обзор данных

In [ ]:
# импорт библиотек
import pandas as pd
pd.options.mode.chained_assignment = None  
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math
from scipy import stats as st
from datetime import datetime, timedelta
from plotly import graph_objects as go

In [ ]:
# чтение файлов с данными и сохранение в df
try:
    data = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
except:
    data = pd.read_csv('logs_exp.csv', sep='\t')

In [ ]:
data.info()
data.sample(5)

На первый взгляд, серьезных проблем с данными нет. Нужно будет проверить их на дубликаты, привести названия столбцов к хорошему стилю и преобразовать данные со временем.

# Предобработка данных

In [ ]:
# проверка таблиц на наличие дубликатов

data.duplicated().sum()

In [ ]:
# удалим дубликаты

data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
# приведем наименования столбцов к хорошему стилю

data = data.rename(columns={'EventName': 'event', 'DeviceIDHash': 'user_id', 
                            'EventTimestamp': 'date_time', 'ExpId': 'group_id'})
data.columns

In [ ]:
# заменим тип данных datetime 

data['date_time'] = pd.to_datetime(data['date_time'], unit='s')

In [ ]:
# добавим поле с датой

data['date'] = pd.to_datetime(data['date_time'].dt.date)
data.head(5)

In [ ]:
# проверим, что пользователи в группах не пересекаются
data.groupby('user_id')['group_id'].nunique().reset_index().query('group_id > 1')

В результате преобработки данных мы удалили явные дубликаты, привели названия столбцов к хорошему стилю и добавили поле с датой. Теперь переходим к изучению и проверке данных.

# Изучение и проверка данных

## Сколько всего событий и пользователей в логе?

Узнаем сколько всего событий в логе. 

In [ ]:
print('Всего событий:', data.shape[0]) 
print('Видов событий:',data['event'].nunique())
print('Всего пользователей:',data['user_id'].nunique())

Итак, в логе 243713 событий, 5 видов событий и 7551 пользователей. Узнаем как распределяется количество событий по пользователям.

In [ ]:
events_by_user = data.groupby('user_id').agg({'event':'count'})
display(events_by_user['event'].describe())

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Распределение количества событий на пользователя')
sns.histplot(events_by_user['event'], bins=50)
plt.xlabel('Количество событий')                   
plt.ylabel('Количество пользователей')             
plt.show()

Получается большинство пользователей совершило примерно 20 событий, притом минимальное количесвто - 1 событие, значит в датафрейме нет пользователей без событий вообще.

## Сколько в среднем событий приходится на пользователя?

In [ ]:
data['date_time'].describe(datetime_is_numeric=True)

Датафрейм содержит данные с 25.07.2019 по 07.08.2019. Построим гистограмму распределения количества пользователей по дате и времени.

In [ ]:
# гистограмма по дате и времени
plt.title('Распределение логов по дате и времени', loc='left')
data['date_time'].hist(bins=14*24,  figsize=(20, 8), alpha=0.7)
plt.show()

На гистограмме видно, что данные на конец июля непольные. В логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Будет целесообразно для дальнейшего анализа использовать данные с 01.08.2019.

Теперь отбросим старые данные и посмотрим сколько событий и пользователей мы потеряли.

In [ ]:
old_logs = data.shape[0]
old_users = data['user_id'].nunique()
print('Было событий:', old_logs)
print('Было пользователей:', old_users)

# отбрасываем старые данные
data = data[data['date_time'] >= '2019-08-01']

print('Стало событий:', data.shape[0])
print('Стало пользователей:', data['user_id'].nunique())

print('Изменение количества логов:', 
      data.shape[0]- old_logs, 
      '({:.1%})'.format((data.shape[0]-old_logs)/old_logs))
print('Изменение количества пользователей:', 
      data['user_id'].nunique()- old_users, 
      '({:.1%})'.format((data['user_id'].nunique()-old_users)/old_users))

Отбросив данные, мы потеряли всего 2826 логов (1,2%) и 17 (0,2%) пользователей, что является незначительным числом по сравнению с общим количеством. Теперь посмотрим на распределение пользователей по экспериментальным группам.

In [ ]:
data.groupby('group_id').agg({'user_id':'nunique'})

Количество пользователей в каждой экспериментальной группе примерно одинаковое.

# Изучение воронки событий

Для начала посмотрим, какие события есть в логах, как часто они встречаются.

In [ ]:
events = (data.
          groupby('event').
          agg({'user_id': 'count'}).
          reset_index().
          rename(columns={'user_id' : 'total_events'}).
          sort_values(by='total_events', ascending=False))
events

In [ ]:
plt.figure(figsize=(15, 5))
ax = sns.barplot(x='total_events', y='event', data=events)
ax.set_title('Частота событий в логах')
ax.set_xlabel('Количество событий') 
plt.show()

Итак, по частоте событий можно сказать следущеее:
- Событие **MainScreenAppear** (просмотр главного экрана) произошло 117328 раз;
- Событие **OffersScreenAppear** (просмотр каталога предложений) произошло 46333 раза;
- Событие **CartScreenAppear** (просмотр карточки товара) произошло 42303 раза;
- Событие **PaymentScreenSuccessful** (просмотр экрана с подтверждением успешной оплаты) произошло 33918 раз;
- Событие **Tutorial** (Обучение) произошло 1005 раз.

Теперь посчитаем, сколько пользователей совершали каждое из этих событий.

In [ ]:
funnel = (data.
          groupby('event').
          agg({'user_id': 'nunique'}).
          reset_index().
          rename(columns={'user_id' : 'users'}).
          sort_values(by='users', ascending=False))
# нам также понадобится доля пользователей, которые хоть раз совершали событие.
funnel['percent'] = round(funnel['users'] / old_users * 100, 2) 
funnel

Порядок событий постепенно становится ясен. 
- Сначала пользователь видит **главную страницу** приложения - 7419 пользователей (98% от общего числа пользователей). 
- Затем он просматривает **каталог предложений** - 4593 пользователей (61% от общего числа)
- Далее он ознакамливается с **карточкой товара** - 3734 пользователя (49% от общего числа)
- И **завершает оплату** - 3539 пользователей (47% от общего числа)

Событие Tutorial прошло всего 840 пользователей (11% от общего числа), поэтому оно никак не вписывается в нашу воронку. Довольно странно, что пользователь сначала завершает оплату в приложении, а потом уже читает инструкцию как им пользоваться. Поэтому его не будем учитывать при расчётах.

In [ ]:
# исключаем из воронки событие Tutorial 
funnel = funnel[funnel['event'] != 'Tutorial']

fig = go.Figure(go.Funnel(x = funnel['users'],
                y = funnel['event'], 
                textposition = "inside",
                textinfo = "value+percent initial",
                opacity = 0.65, 
                marker = {"color": ["#ffd800", "#ff4900", "#0076ff", "#00ffa6"], 
                "color": ["yellow", "red", "blue", "green"]})
    )
fig.show()

In [ ]:
# Посчитаем долю пользователей, переходящих на следующий шаг воронки (колонка percent_of_previous)
funnel['percent_of_previous'] = round(funnel['users'] / funnel['users'].shift() * 100, 2)
funnel   

Судю по всему меньше всего пользователей перешли с этапа просмотра главного экрана на этап просмотра каталога товаров. Возможно, это связано с неудобным интерфейсом или некорректной работой этой функции приложения у некоторых пользователей.

По графику видно, что целых 38% пользователей перестают пользоваться приложением после первого шага. И только 48% от общего количества доходят до этапа успешной оплаты товара. Такие изменения могут быть связаны с неудобным и интуитивно непонятным интерфейсом или же техническими неполадками (например, если эта функция некорректно работает на некоторых устройствах). 

## Изучение результатов эксперимента


Сначала узнаем сколько пользователей в каждой экспериментальной группе.

In [ ]:
data.groupby('group_id').agg(count=('user_id', 'nunique'))

Количество пользователей в группах примерно одинаковое. 

В рамках исследования мы будем сравнивать:

- Контрольные группы со старыми шрифтами - 246 с 247;
- Каждую из контрольных групп по отдельности с экспериментальной (246 с 248 и 247 с 248);
- Объединенную контрольную группу и экспериментальную (246 + 247 с 248).

Для удобства напишем функцию, которая, проведя статистический тест, позволит определить, различаются ли доли генеральной совокупности. Всего у нас 4 события, а значит 4 А/A-теста и 12 А/B-тестов. Для корректировки уровня значимости при множественных сравнениях будем использовать поправку Бонферонни (уровень значимости будем делить на количество сравнений).
Аргументы функции:
- **df1**, **df2** - датафреймы с логами
- **event** - событие
- **alpfa** - критический уровень статистической значимости
- **n** - поправка Боннферони для критического уровня статистической значимости

In [ ]:
def z_test(df1, df2, event, alpha, n):
     
    # критический уровень статистической значимости c поправкой Бонферрони
    bonferroni_alpha = alpha / n
 
    # число пользователей в группе 1 и группе 2:
    n_users = np.array([df1['user_id'].nunique(), 
                        df2['user_id'].nunique()])

    # число пользователей, совершивших событие в группе 1 и группе 2
    success = np.array([df1[df1['event'] == event]['user_id'].nunique(), 
                        df2[df2['event'] == event]['user_id'].nunique()])

    # пропорция успехов в первой группе:
    p1 = success[0]/n_users[0]
    # пропорция успехов во второй группе:
    p2 = success[1]/n_users[1]
    
    # пропорция успехов в комбинированном датасете:
    p_combined = (success[0] + success[1]) / (n_users[0] + n_users[1])

    # разница пропорций в датасетах
    difference = p1 - p2 

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference /  np.sqrt(p_combined * (1 - p_combined) * (1/n_users[0] + 1/n_users[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)  

    p_value = (1 - distr.cdf(abs(z_value))) * 2   # тест двусторонний, удваиваем результат
    
    print('Событие:', event)
    print('p-значение: ', p_value)

    if p_value < bonferroni_alpha:
        print('Отвергаем нулевую гипотезу: между долями есть разница')
    else:
        print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')

Введем основную и альтернативную гипотезы:
- **H0** - доли уникальных посетителей побывавших на этапе воронки одинаковы
- **H1** - доли уникальных посетителей побывавших на этапе воронки разные

### А/А-тест

У нас есть 2 контрольные группы для А/А-эксперимента, чтобы проверить корректность всех механизмов и расчётов. Проверим, находят ли статистические критерии разницу между выборками 246 и 247.

In [ ]:
for event in funnel['event'].unique():
    z_test(data[data['group_id'] == 246], data[data['group_id'] == 247], event,.05, 4)
    print()

Между группами 246 и 247 ни по одному событию нет статистически достоверного отличия при выбранном уровне значимости, значит можно приступать к А/B тестированию.

### A/A/B-тест

#### Группы 246 и 248.

Проверим, есть ли статистически значимая разница между контрольными группами 246 и 248.

In [ ]:
for event in funnel['event'].unique():
    z_test(data[data['group_id'] == 246], data[data['group_id'] == 248], event, .05, 12)
    print()

У нас нет оснований считать контрольные группы 246 и 248 разными при заданном уровне статистически значимой.

#### Группы 247 и 248.

Теперт проверим разницу между группами 247 и 248.

In [ ]:
for event in funnel['event'].unique():
    z_test(data[data['group_id'] == 247], data[data['group_id'] == 248], event,.05,12)
    print()

Аналогично и с группой 247 по сравнению с экспериментальной группой 248 - статистически значимых различий нет.

#### Объединенная группа 246+247 и экспериментальная группа 248.

In [ ]:
for event in funnel['event'].unique():
    z_test(data[data['group_id'] != 248], data[data['group_id'] == 248], event,.05,12)
    print()

Снова никакой статистически значимой разницы разницы. 

Из всех проведенных нами тестов следует вывод о том, что изменение шрифтов во всем приложении никак не повлияло на поведение пользователей.

# Выводы

В ходе исследования мы проанализировали поведение покупателей на основании логов пользователей, а так же, результаты А/А/В-теста. 

Резюмируя результаты проведенного анализа:

**Воронка событий:**
- Главную страницу (MainScreenAppear) увидели 7419 пользователей (100% от общего числа пользователей);
- Страницу товара (OffersScreenAppear) просмотрели 4593 пользователей (62% от общего числа);
- Карточку товара (CartScreenAppear) просмотрели 3734 пользователя (50% от общего числа);
- Завершили оплату (PaymentScreenSuccessful) 3539 пользователей (48% от общего числа).
- Прохождение обучения (Tutorial) мы исключили из анализа, так как его прошло всего 840 пользователей (11% от общего числа), и оно никак не вписывается в воронку событий. 

Больше всего пользователей приложение потеряло после первого шага (более 38%), примерно 9% пользователей на следующем шаге и около 2% при переходе на последний шаг. Таким образом, примерно 48% пользователей дошло до этапа оплаты заказа. 

**A/A/B-эксперимент:**
Также мы проанализировали результаты А/А/В-эксперимента, суть которого заключалась в изменении шрифта во всем приложении. В эксперименте были ипользованы логи событий за неделю (с 01/08/2019 по 07/08/2019).

В эксперименте участвовали пользователи, разделенные на три группы (две контрольные - 246 и 247 и одна экспериментальная - 248).

По результатам A/A/B теста можно сделать вывод, что изменение шрифтов во всем приложении никак не повлияло на поведение пользователей, так как статистически значимой разницы между группами не обнаружилось.

Таким образом, изменение шрифтов во всём приложении на поведение пользователей не повлияло.